<a href="https://colab.research.google.com/github/Taeye-Kwack/Predict_CO2/blob/main/Predict_CO2_by_RandomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This is the code that make model through several ML methods

import numpy as np
import pandas as pd

fpath = '/content/drive/MyDrive/'
all = np.load(fpath + 'Allvariables_new_2020.npy') # Meteorological data and Satellite observation data in the same time
all = pd.DataFrame(all)
all.columns = ['t2m','sp','u10','v10','u100','v100','u','v','w','co','co_time','no2','co2_time','hour','lat','lon','sif','co2']
# Sun-induced fluorescence (SIF) is NaN in the ocean, so we can devide the satellite data whether sif is NaN or not.
land = all[~np.isnan(all.sif)] 
land = land[['t2m', 'sp', 'u10', 'v10', 'u100', 'v100', 'u', 'v', 'w', 'co','co_time', 'no2', 'lat', 'lon', 'sif', 'co2']]
ocean = all[np.isnan(all.sif)] 
ocean = ocean[['t2m', 'sp', 'u10', 'v10', 'u100', 'v100', 'u', 'v', 'w', 'co','co_time', 'no2', 'lat', 'lon', 'co2']]

In [ ]:
all

In [ ]:
!pip install Cartopy==0.17.0
import cartopy.crs as ccrs
import cartopy

In [ ]:
import matplotlib.pyplot as plt  
#ax = plt.figure()
#ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=180)) # There is runtime error when do sth about Cartopy library
#ax.coastlines(zorder=1)
plt.subplot(211)
plt.scatter(land['lon'],land['lat'], c=land['co2'],s=0.1,vmin=400,vmax=420, cmap = 'jet')
xticks = range(-180, 181, 60)
#ax.set_xticks(xticks, crs=ccrs.PlateCarree())
yticks = range(-90, 91, 30)
#ax.set_yticks(yticks, crs=ccrs.PlateCarree())
plt.colorbar(label = 'XCO2 (ppm)',shrink=0.7)
plt.title('CO2 satellite data over land in 2020')
plt.xlabel('longitude')
plt.ylabel('latitude')

plt.subplot(212)
plt.scatter(ocean['lon'],ocean['lat'], c=ocean['co2'],s=0.1,vmin=400,vmax=420, cmap = 'jet')
xticks = range(-180, 181, 60)
#ax.set_xticks(xticks, crs=ccrs.PlateCarree())
yticks = range(-90, 91, 30)
#ax.set_yticks(yticks, crs=ccrs.PlateCarree())
plt.colorbar(label = 'XCO2 (ppm)',shrink=0.7)
plt.title('CO2 satellite data over land in 2020')
plt.xlabel('longitude')
plt.ylabel('latitude')

plt.tight_layout()

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from scipy.stats import linregress
from scipy import stats

land_np = np.array(land)
X = land_np[:,0:-1]
y = land_np[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

land_model = RandomForestRegressor()
land_model.fit(X_train, y_train)

y_predict = land_model.predict(X_test)
print('Land_model score :',land_model.score(X_test, y_test))

m1 = y_test
m2 = y_predict
m2 = m2.reshape(-1)
values = np.vstack([m1, m2])
kernel = stats.gaussian_kde(values)(values)
fig = plt.figure()
ax = fig.add_subplot(111)
plt.scatter(y_test,y_predict,c=kernel,s=1,edgecolors='')
plt.ylabel('Predicted CO2 (ppm)')
plt.xlabel('Observed CO2 (ppm)')
plt.xlim(390,430)
plt.ylim(390,430)
plt.xticks(np.arange(390,440, step=10))
plt.yticks(np.arange(390,440, step=10))
ax.set_aspect('equal', adjustable='box')

rmse = (mean_squared_error(y_test, y_predict))**0.5
co2lin = linregress(m1, m2)

plt.title('Comparison of observed CO2 by OCO2 and predicted CO2 using RF (land)')
plt.text(420,410,"y = %.2f x + %.2f \nR-square = %.2f \nRMSE = %.2f" % (co2lin[0],co2lin[1], r2_score(y_test,y_predict), rmse), fontsize=10)

xlin = np.linspace(390, 430, 100)
ylin = xlin * co2lin[0]+ co2lin[1]
plt.plot(xlin, ylin, linewidth='1')

xlin = np.linspace(390, 430, 100)
ylin = xlin
plt.plot(xlin, ylin, linewidth='1',linestyle = ':')

In [ ]:
'''
ocean_np = np.array(ocean)
X = ocean_np[:,0:-1]
y = ocean_np[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

ocean_model = RandomForestRegressor()
ocean_model.fit(X_train, y_train)
'''
y_predict = ocean_model.predict(X_test)
print('ocean_model score :',ocean_model.score(X_test, y_test))

m1 = y_test
m2 = y_predict
m2 = m2.reshape(-1)
values = np.vstack([m1, m2])
kernel = stats.gaussian_kde(values)(values)
fig = plt.figure()
ax = fig.add_subplot(111)
plt.scatter(y_test,y_predict,c=kernel,s=1,edgecolors='')
plt.ylabel('Predicted CO2 (ppm)')
plt.xlabel('Observed CO2 (ppm)')
plt.xlim(390,430)
plt.ylim(390,430)
plt.xticks(np.arange(390,440, step=10))
plt.yticks(np.arange(390,440, step=10))
ax.set_aspect('equal', adjustable='box')

rmse = (mean_squared_error(y_test, y_predict))**0.5
co2lin = linregress(m1, m2)

plt.title('Comparison of observed CO2 by OCO2 and predicted CO2 using RF (ocean)')
plt.text(420,410,"y = %.2f x + %.2f \nR-square = %.2f \nRMSE = %.2f" % (co2lin[0],co2lin[1], r2_score(y_test,y_predict), rmse), fontsize=10)

xlin = np.linspace(390, 430, 100)
ylin = xlin * co2lin[0]+ co2lin[1]
plt.plot(xlin, ylin, linewidth='1')

xlin = np.linspace(390, 430, 100)
ylin = xlin
plt.plot(xlin, ylin, linewidth='1',linestyle = ':')

In [ ]:
import joblib
joblib.dump(land_model, './land_model.pkl')
joblib.dump(ocean_model, './ocean_model.pkl')